In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as st
import sys 
import datetime as dt
import time
import gc
gc.collect()

0

In [3]:
from helper.parallel_util import ParallelUtil
from helper.parsers import Utils
from helper.file_mapping import FILE_MAPPINGS
from helper.imdb_sqllite_db import IMDBSQLLite
import config


LESSON_DATA_FOLDER = config.IMDB_DATA_FOLDER
imdb_db = IMDBSQLLite();
imdbConn, imdbCurs = imdb_db.get()

IMDB SQLLite Database: ../data/imdb.sqllite


In [ ]:
#putil = ParallelUtil('./data/parallel.config.tsv')

In [6]:
title_files =[]
LIMIT = 100000

for x in range(0, int(imdb_db.rowCount('merged_title_ratings')['count']/LIMIT) + 2):
    title_files.append({'index':x, 'limit':LIMIT, 'offset': LIMIT * x, 
                        'file':''.join([LESSON_DATA_FOLDER, 'titles.' , str(x), '.json'])})

#title_files[0]    
len(title_files)/2

25.0

In [ ]:
import prep_data;
for title in title_files[:25]:    
    print ('Working on title : {}'.format(title))
    prep_data.prep_title (title)
    gc.collect()
    

Working on title : {'limit': 100000, 'file': '../data/titles.0.json', 'index': 0, 'offset': 0}
Working on title : {'limit': 100000, 'file': '../data/titles.1.json', 'index': 1, 'offset': 100000}
Working on title : {'limit': 100000, 'file': '../data/titles.2.json', 'index': 2, 'offset': 200000}
Working on title : {'limit': 100000, 'file': '../data/titles.3.json', 'index': 3, 'offset': 300000}
Working on title : {'limit': 100000, 'file': '../data/titles.4.json', 'index': 4, 'offset': 400000}
Working on title : {'limit': 100000, 'file': '../data/titles.5.json', 'index': 5, 'offset': 500000}
Working on title : {'limit': 100000, 'file': '../data/titles.6.json', 'index': 6, 'offset': 600000}
Working on title : {'limit': 100000, 'file': '../data/titles.7.json', 'index': 7, 'offset': 700000}
Working on title : {'limit': 100000, 'file': '../data/titles.8.json', 'index': 8, 'offset': 800000}
Working on title : {'limit': 100000, 'file': '../data/titles.9.json', 'index': 9, 'offset': 900000}
Worki